In [33]:
# Load LSTM network and generate text
import sys
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import h5py
import re
import tensorflow as tf
import pandas as pd
from pprint import pprint

In [2]:
import platform
platform.architecture()[0]

'64bit'

In [34]:
mov_lines_file=open("/home/kavin/Silo/College Work/AI/DataSets/cornell_movie_dialogs_corpus/cornell movie-dialogs corpus/movie_lines.txt",'r')

In [35]:
mov_lines=mov_lines_file.readlines()

In [36]:
dialogues=[]
for mov_line in mov_lines:
    matched=re.match('.* \+\+\+\$\+\+\+ (.+)', mov_line)
    dialogues.append(matched.group(1))

In [ ]:
dialogues[1]

In [37]:
tokenized_dialogues=[]
for line in dialogues:
    tokenized_dialogues.append(filter(None,re.split(' |!|:|;|\?|"|\-\-|\*|<.*?>|</.*?>', line.lower())))

In [ ]:
tokenized_dialogues[0]

In [38]:
cleaned_dialogues=[]
for line in tokenized_dialogues:
    line_list=[]
    for token in line:
        try:
            matched=re.match('([\*|\'|_|\-|\[]|)(.*)([\*|\'|_|\-|\]]|)', token)
            line_list.append(matched.group(2))
        except:
            pass
    cleaned_dialogues.append(line_list)

In [ ]:
cleaned_dialogues[0]

In [39]:
all_dialogues=[]
for line in cleaned_dialogues:
    all_dialogues+=line
tokens=pd.Series(all_dialogues).unique()

In [40]:
bag_of_words={}
index_to_words={}
index=1
for word in tokens:
    bag_of_words[word]=index
    index_to_words[index]=word
    index+=1
#n_chars = len(tokenized)
n_vocab = len(tokens)

In [54]:
bag_of_words["hello"]

2863

In [ ]:
cleaned_dialogues[:10]

In [41]:
indexed_dialogues=[]
for line in cleaned_dialogues:
    line_list=[]
    for token in line:
        try:
            line_list.append(bag_of_words[token])
        except:
            line_list.append('0')
    indexed_dialogues.append(line_list)

In [ ]:
len(indexed_dialogues[1])

In [42]:
dialogues_dict={}
line_number=0
for line in indexed_dialogues:
    dialogues_dict[line_number]=line
    line_number+=1

In [43]:
all_keys=dialogues_dict.keys()

In [44]:
reply_keys=[]
response_keys=[]
for key in all_keys:
    if(key%2==0):
        reply_keys.append(key)
    else:
        response_keys.append(key)

In [ ]:
reply_index=0
response_index=1


In [ ]:
dialogues_df.head()

In [45]:
all_lines=[]
all_responses=[]

In [46]:
all_lines=list(np.array(indexed_dialogues)[reply_keys])
all_responses=list(np.array(indexed_dialogues)[response_keys])

In [ ]:
total_length=0
for line in all_lines:
    total_length+=len(line)
print total_length/len(all_lines)

In [ ]:
total_length=0
for line in all_responses:
    total_length+=len(line)
print total_length/len(all_responses)

In [60]:
len(all_lines)

152357

In [61]:
sample_lines=[]
sample_responses=[]
for line in all_lines[20000:]:
    sample_lines+=line
    if(len(sample_lines)>=10000):
        break
for line in all_responses[20000:]:
    sample_responses+=line
    if(len(sample_responses)>=10000):
        break

In [31]:
len(sample_lines)
len(sample_responses)

1013

In [62]:
seq_length = 10
dataX = []
dataY = []
input_lines=[]
output_line=[]
for i in range(0, len(sample_lines)-seq_length, 1):
    input_line=sample_lines[i:i+seq_length]
    output_line=sample_responses[i+seq_length]
    dataX.append(input_line)
    dataY.append(output_line)
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

Total Patterns:  9991


In [20]:
len(dataY)

990

In [18]:
len(max(dataX, key=len))

84774

In [63]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (len(dataX), 10, 1))
    # normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [65]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [66]:
# define the checkpoint
filepath="Donna Files/text_gen_weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=1000, batch_size=128, callbacks=callbacks_list)

Epoch 1/1000
9991/9991 [==============================] - 20s - loss: 8.2459    
Epoch 2/1000
9991/9991 [==============================] - 19s - loss: 6.9853    
Epoch 3/1000
9991/9991 [==============================] - 19s - loss: 6.7995    
Epoch 4/1000
9991/9991 [==============================] - 19s - loss: 6.7146    
Epoch 5/1000
9991/9991 [==============================] - 19s - loss: 6.6993    
Epoch 6/1000
9991/9991 [==============================] - 19s - loss: 6.7001    
Epoch 7/1000
9991/9991 [==============================] - 19s - loss: 6.6956    
Epoch 8/1000
9991/9991 [==============================] - 19s - loss: 6.7004    
Epoch 9/1000
9991/9991 [==============================] - 19s - loss: 6.7010    
Epoch 10/1000
9991/9991 [==============================] - 19s - loss: 6.7034    
Epoch 11/1000
9991/9991 [==============================] - 19s - loss: 6.6979    
Epoch 12/1000
9991/9991 [==============================] - 19s - loss: 6.7042    
Epoch 13/1000
9991/9991 [

9991/9991 [==============================] - 19s - loss: 6.7065    
Epoch 43/1000
9991/9991 [==============================] - 19s - loss: 6.7059    
Epoch 44/1000
9991/9991 [==============================] - 19s - loss: 6.6980    
Epoch 45/1000
9991/9991 [==============================] - 19s - loss: 6.7021    
Epoch 46/1000
9991/9991 [==============================] - 19s - loss: 6.6977    
Epoch 47/1000
9991/9991 [==============================] - 19s - loss: 6.6996    
Epoch 48/1000
9991/9991 [==============================] - 19s - loss: 6.7010    
Epoch 49/1000
9991/9991 [==============================] - 19s - loss: 6.6956    
Epoch 50/1000
9991/9991 [==============================] - 19s - loss: 6.6939    
Epoch 51/1000
9991/9991 [==============================] - 19s - loss: 6.6892    
Epoch 52/1000
9991/9991 [==============================] - 19s - loss: 6.6941    
Epoch 53/1000
9991/9991 [==============================] - 19s - loss: 6.6853    
Epoch 54/1000
9991/9991 [=====

9991/9991 [==============================] - 19s - loss: 6.6392    
Epoch 81/1000
9991/9991 [==============================] - 19s - loss: 6.6382    
Epoch 82/1000
9991/9991 [==============================] - 19s - loss: 6.6336    
Epoch 83/1000
9991/9991 [==============================] - 19s - loss: 6.6334    
Epoch 84/1000
9991/9991 [==============================] - 19s - loss: 6.6287    
Epoch 85/1000
9991/9991 [==============================] - 19s - loss: 6.6293    
Epoch 86/1000
9991/9991 [==============================] - 19s - loss: 6.6862    
Epoch 87/1000
9991/9991 [==============================] - 19s - loss: 6.6428    
Epoch 88/1000
9991/9991 [==============================] - 20s - loss: 6.6397    
Epoch 89/1000
9991/9991 [==============================] - 20s - loss: 6.6307    
Epoch 90/1000
9991/9991 [==============================] - 20s - loss: 6.6440    
Epoch 91/1000
9991/9991 [==============================] - 22s - loss: 6.6294    
Epoch 92/1000
9991/9991 [=====

9991/9991 [==============================] - 19s - loss: 6.2124    
Epoch 114/1000
9991/9991 [==============================] - 19s - loss: 6.1908    
Epoch 115/1000
9991/9991 [==============================] - 19s - loss: 6.1637    
Epoch 116/1000
9991/9991 [==============================] - 19s - loss: 6.1486    
Epoch 117/1000
9991/9991 [==============================] - 19s - loss: 6.1191    
Epoch 118/1000
9991/9991 [==============================] - 19s - loss: 6.0955    
Epoch 119/1000
9991/9991 [==============================] - 19s - loss: 6.0748    
Epoch 120/1000
9991/9991 [==============================] - 19s - loss: 6.0501    
Epoch 121/1000
9991/9991 [==============================] - 20s - loss: 6.0494    
Epoch 122/1000
9991/9991 [==============================] - 19s - loss: 6.0094    
Epoch 123/1000
9991/9991 [==============================] - 19s - loss: 5.9775    
Epoch 124/1000
9991/9991 [==============================] - 20s - loss: 5.9550    
Epoch 125/1000
9991

9991/9991 [==============================] - 19s - loss: 5.4913    
Epoch 144/1000
9991/9991 [==============================] - 19s - loss: 5.4833    
Epoch 145/1000
9991/9991 [==============================] - 19s - loss: 5.4554    
Epoch 146/1000
9991/9991 [==============================] - 19s - loss: 5.4308    
Epoch 147/1000
9991/9991 [==============================] - 19s - loss: 5.4276    
Epoch 148/1000
9991/9991 [==============================] - 19s - loss: 5.3972    
Epoch 149/1000
9991/9991 [==============================] - 19s - loss: 5.3830    
Epoch 150/1000
9991/9991 [==============================] - 19s - loss: 5.3444    
Epoch 151/1000
9991/9991 [==============================] - 19s - loss: 5.3240    
Epoch 152/1000
9991/9991 [==============================] - 19s - loss: 5.3305    
Epoch 153/1000
9991/9991 [==============================] - 19s - loss: 5.2895    
Epoch 154/1000
9991/9991 [==============================] - 19s - loss: 5.2518    
Epoch 155/1000
9991

...

9991/9991 [==============================] - 26s - loss: 1.4234    
Epoch 809/1000
9991/9991 [==============================] - 26s - loss: 1.4363    
Epoch 810/1000
9991/9991 [==============================] - 26s - loss: 1.4512    
Epoch 811/1000
9991/9991 [==============================] - 26s - loss: 1.3482    
Epoch 812/1000
9991/9991 [==============================] - 26s - loss: 1.3882    
Epoch 813/1000
9991/9991 [==============================] - 26s - loss: 1.5809    
Epoch 814/1000
9991/9991 [==============================] - 26s - loss: 1.4356    
Epoch 815/1000
9991/9991 [==============================] - 26s - loss: 1.3550    
Epoch 816/1000
9991/9991 [==============================] - 26s - loss: 1.3983    
Epoch 817/1000
9991/9991 [==============================] - 26s - loss: 1.4824    
Epoch 818/1000
9991/9991 [==============================] - 26s - loss: 1.5531    
Epoch 819/1000
9991/9991 [==============================] - 26s - loss: 1.4535    
Epoch 820/1000
9991

9991/9991 [==============================] - 25s - loss: 1.4294    
Epoch 850/1000
9991/9991 [==============================] - 26s - loss: 1.3116    
Epoch 851/1000
9991/9991 [==============================] - 26s - loss: 1.6494    
Epoch 852/1000
9991/9991 [==============================] - 26s - loss: 1.2925    
Epoch 853/1000
9991/9991 [==============================] - 26s - loss: 1.2526    
Epoch 854/1000
9991/9991 [==============================] - 26s - loss: 1.2264    
Epoch 855/1000
9991/9991 [==============================] - 25s - loss: 1.2533    
Epoch 856/1000
9991/9991 [==============================] - 25s - loss: 1.2905    
Epoch 857/1000
9991/9991 [==============================] - 26s - loss: 1.2260    
Epoch 858/1000
9991/9991 [==============================] - 25s - loss: 1.2865    
Epoch 859/1000
9991/9991 [==============================] - 25s - loss: 1.2666    
Epoch 860/1000
9991/9991 [==============================] - 25s - loss: 1.3232    
Epoch 861/1000
9991

9991/9991 [==============================] - 19s - loss: 1.4278    
Epoch 892/1000
9991/9991 [==============================] - 19s - loss: 1.2198    
Epoch 893/1000
9991/9991 [==============================] - 19s - loss: 1.3276    
Epoch 894/1000
9991/9991 [==============================] - 19s - loss: 1.2171    
Epoch 895/1000
9991/9991 [==============================] - 19s - loss: 1.2471    
Epoch 896/1000
9991/9991 [==============================] - 19s - loss: 1.3532    
Epoch 897/1000
9991/9991 [==============================] - 19s - loss: 1.2714    
Epoch 898/1000
9991/9991 [==============================] - 19s - loss: 1.1854    
Epoch 899/1000
9991/9991 [==============================] - 19s - loss: 1.1958    
Epoch 900/1000
9991/9991 [==============================] - 19s - loss: 1.2509    
Epoch 901/1000
9991/9991 [==============================] - 19s - loss: 1.3232    
Epoch 902/1000
9991/9991 [==============================] - 19s - loss: 1.1880    
Epoch 903/1000
9991

9991/9991 [==============================] - 19s - loss: 1.0635    
Epoch 936/1000
9991/9991 [==============================] - 19s - loss: 1.1948    
Epoch 937/1000
9991/9991 [==============================] - 19s - loss: 1.1087    
Epoch 938/1000
9991/9991 [==============================] - 19s - loss: 1.1148    
Epoch 939/1000
9991/9991 [==============================] - 19s - loss: 1.2721    
Epoch 940/1000
9991/9991 [==============================] - 19s - loss: 1.1725    
Epoch 941/1000
9991/9991 [==============================] - 19s - loss: 1.0982    
Epoch 942/1000
9991/9991 [==============================] - 19s - loss: 1.1266    
Epoch 943/1000
9991/9991 [==============================] - 19s - loss: 1.3546    
Epoch 944/1000
9991/9991 [==============================] - 19s - loss: 1.0884    
Epoch 945/1000
9991/9991 [==============================] - 19s - loss: 1.1099    
Epoch 946/1000
9991/9991 [==============================] - 19s - loss: 1.1512    
Epoch 947/1000
9991

9991/9991 [==============================] - 19s - loss: 1.0259    
Epoch 978/1000
9991/9991 [==============================] - 19s - loss: 1.0444    
Epoch 979/1000
9991/9991 [==============================] - 19s - loss: 1.0758    
Epoch 980/1000
9991/9991 [==============================] - 19s - loss: 1.0286    
Epoch 981/1000
9991/9991 [==============================] - 19s - loss: 1.0678    
Epoch 982/1000
9991/9991 [==============================] - 19s - loss: 1.1017    
Epoch 983/1000
9991/9991 [==============================] - 19s - loss: 1.3042    
Epoch 984/1000
9991/9991 [==============================] - 19s - loss: 1.0427    
Epoch 985/1000
9991/9991 [==============================] - 19s - loss: 1.0048    
Epoch 986/1000
9991/9991 [==============================] - 19s - loss: 1.0325    
Epoch 987/1000
9991/9991 [==============================] - 19s - loss: 1.0139    
Epoch 988/1000
9991/9991 [==============================] - 19s - loss: 1.0911    
Epoch 989/1000
9991

In [59]:
# load the network weights
filename = "Donna Files/weights-improvement-989-0.1795.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [53]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = ["Hello",".","How", "are", "you"]
print "Seed:"
print "\"", ''.join([str(value) for value in pattern]), "\""
indexed_input=[]
([indexed_input.append(bag_of_words[value.lower()]) for value in pattern])


Seed:
" Hello.Howareyou "


[None, None, None, None, None]

In [54]:
pattern=indexed_input

In [55]:
pattern+=list(np.zeros(seq_length-len(pattern)))

In [61]:
np.zeros(seq_length-len(pattern))

array([ 0.,  0.,  0.,  0.,  0.,  0.])

In [67]:
# generate characters
for i in range(10):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = index_to_words[index]
	#seq_in = [index_to_words[value] for value in pattern]
	sys.stdout.write(result+" ")
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print "\nDone."

when a him keen. finally have i dog. lecture i 
Done.
